In [1]:
import pandas as pd
import time
import numpy as np
from nba_api.stats.endpoints import playercareerstats
import nba_api.stats.endpoints as nbaapi
from nba_api.stats.endpoints import leaguegamefinder
import psycopg2
import re
import allintwo

In [2]:
# from nba_api.stats.endpoints import leaguegamefinder

seasons = []
for i in list(range(1983,2024)):
    seasons.append(str(i)+'-'+str(i+1)[2:])
seasons = seasons[::-1]

# gamesl = []
# for i in seasons:
#     gamefinder = leaguegamefinder.LeagueGameFinder(league_id_nullable='00', season_type_nullable="Regular Season",season_nullable=i).get_data_frames()[0]
#     gamesl.append(gamefinder)
#     time.sleep(1)
# gamehistory = pd.concat(gamesl,axis=0)
# gamehistory['GAME_DATE'] =  pd.to_datetime(gamehistory['GAME_DATE'])

# playersl = []
# for i in seasons:
#     playerfinder = nbaapi.leaguedashplayerbiostats.LeagueDashPlayerBioStats(season=i).get_data_frames()[0]
#     playerfinder['season']=i
#     playersl.append(playerfinder)
#     time.sleep(.5)
# allplayers = pd.concat(playersl,axis=0)

In [3]:
## Takes only 1:20 to pull
gamesl = []
for i in seasons:
    gamefinder = leaguegamefinder.LeagueGameFinder(season_nullable=i).get_data_frames()[0]
    gamesl.append(gamefinder)
    time.sleep(1)
gamehistory = pd.concat(gamesl,axis=0)
gamehistory['GAME_DATE'] =  pd.to_datetime(gamehistory['GAME_DATE'])

## Takes only 1:07
playersl = []
for i in seasons:
    playerfinder = nbaapi.leaguedashplayerbiostats.LeagueDashPlayerBioStats(season=i).get_data_frames()[0]
    playerfinder['season']=i
    playersl.append(playerfinder)
    time.sleep(.5)
allplayers = pd.concat(playersl,axis=0)

C:\Users\ajwin\AppData\Local\Temp\ipykernel_117364\53510150.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  gamehistory = pd.concat(gamesl,axis=0)
C:\Users\ajwin\AppData\Local\Temp\ipykernel_117364\53510150.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  allplayers = pd.concat(playersl,axis=0)


In [7]:
conn = allintwo.connect_to_rds('thebigone', 'ajwin', 'CharlesBark!23', 'nba-rds-instance.c9wwc0ukkiu5.us-east-1.rds.amazonaws.com')

### DROP TABLES
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS mastergames;")
cursor.execute("DROP TABLE IF EXISTS masterplayers;")

conn.commit()

gamehistory = allintwo.clean_column_names(gamehistory)
allplayers = allintwo.clean_column_names(allplayers)

allintwo.create_table(conn,'mastergames',gamehistory)
allintwo.create_table(conn,'masterplayers',allplayers)

Connected to RDS PostgreSQL database


In [9]:
# Call the function to insert the DataFrame data into the RDS table
allintwo.insert_dataframe_to_rds(conn,
    gamehistory,
    table_name='mastergames'
)


InterfaceError: connection already closed